In [2]:
import pandas as pd
import numpy as np
import plotly as plt

import math
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import geopandas as gpd

import plotly.io as pio
import plotly.graph_objects as go   
import country_converter as coco
from plotly.subplots import make_subplots

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [3]:
# Import datasets
freight = pd.read_csv("Data_Sets/avia_gor_nl__custom_3564729_monthly_linear.csv")
passengers = pd.read_csv("Data_Sets/avia_par_nl__custom_3564728_monthly_linear.csv")

# Setting up special cleaning list
country_code_fix = {'EL':'GR','AN':'BQ'}

def top_n_year(df,n,year):
    # This function has a dataframe, a number of entries and a year as inputs and 
    # as output the dataframe filtered for the top n countries on a specific year
    df[["airp_country_1","airp_code_1","airp_country_2","airp_code_2"]] = df.airp_pr.str.split("_",expand=True)
    df[["Year","Month"]] = df.TIME_PERIOD.str.split("-",expand=True)
    topn = df.groupby(['Year',"airp_country_2"]).sum()
    topn = topn.loc[year,:].sort_values("OBS_VALUE", ascending= False).head(n).reset_index()
    cleaned = df[df["airp_country_2"].isin(topn["airp_country_2"])]
    return cleaned

def codes_correction(df,ISO_2):
    # This function has a data frame and a ISO2 countries list and outputs the same 
    # dataframe adding a ISO3 code and the country name for each unique
    df[ISO_2] = df[ISO_2].replace(to_replace= country_code_fix)
    df['iso_3_country'] = coco.convert(names = df[ISO_2], to= 'ISO3')
    df['country_2_name'] = coco.convert(names = df[ISO_2], to= 'name_short')
    return df

# Call defined functions for the 2 datasets
top_passengers = top_n_year(passengers,5,'2019')
top_cargo = top_n_year(freight,5,'2019')

In [53]:
#COVID PART 
filepath = 'Data_Sets/owid-covid-data.csv'
covid_data = pd.read_csv(filepath, delimiter=',')

# Select the columns of interest from the original dataset and filter it, also expand the date to accomodate year and month
columns_of_interest = ['new_cases_smoothed_per_million','people_fully_vaccinated_per_hundred','date','location']
filtered_covid_data = covid_data[columns_of_interest]
filtered_covid_data[['Year','Month','Day']] = filtered_covid_data.date.str.split("-",expand=True)

# Selecting the countries of interest for cargo and passenger transport
cargo_countries_of_interest =['China','United States','United Arab Emirates','Brazil','Qatar']
passenger_countries_of_interest = ['United Kingdom','Spain','Germany','Italy','United States']

# Group by year, month and location to calculate the mean for cargo transport
cargo_filtered_covid_data = filtered_covid_data[filtered_covid_data['location'].isin(cargo_countries_of_interest)]
cargo_filtered_covid_data = cargo_filtered_covid_data.groupby(['location','Year','Month']).mean()

# Resetting the index and creating new collum with Year and Month 
cargo_filtered_covid_data = cargo_filtered_covid_data.reset_index()
cargo_filtered_covid_data['date'] = cargo_filtered_covid_data['Year'] + ['-'] + cargo_filtered_covid_data['Month'] 

# Group by year, month and location to calculate the mean for cargo transport
passenger_filtered_covid_data = filtered_covid_data[filtered_covid_data['location'].isin(passenger_countries_of_interest)]
passenger_filtered_covid_data = passenger_filtered_covid_data.groupby(['location','Year','Month']).mean()

#Resetting the index and creating new collum with Year and Month 
passenger_filtered_covid_data = passenger_filtered_covid_data.reset_index()
passenger_filtered_covid_data['date'] = passenger_filtered_covid_data['Year'] + ['-'] + passenger_filtered_covid_data['Month']

c:\Users\s167786\Anaconda3\envs\TIL6022\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [39]:

passengers_pre_merge = top_passengers.groupby(['airp_country_2','TIME_PERIOD','Month']).sum()
passengers_pre_merge = passengers_pre_merge.reset_index()
passengers_pre_merge = passengers_pre_merge[passengers_pre_merge['TIME_PERIOD'].between('2020-01', '2022-07')]
passengers_pre_merge = codes_correction(passengers_pre_merge,'airp_country_2')

cargo_pre_merge = top_cargo.groupby(['airp_country_2','TIME_PERIOD','Month']).sum()
cargo_pre_merge = cargo_pre_merge.reset_index()
cargo_pre_merge = cargo_pre_merge[cargo_pre_merge['TIME_PERIOD'].between('2020-01', '2022-07')]
cargo_pre_merge = codes_correction(cargo_pre_merge,'airp_country_2')

In [52]:
#cargo_filtered_covid_data.set_index('date', inplace = True)
cargo_filtered_covid_data
#cargo_filtered_covid_data = cargo_filtered_covid_data.drop(['2022-08','2022-09','2022-10'])



,location,Year,Month,new_cases_smoothed_per_million,people_fully_vaccinated_per_hundred
date,,,,,
2020-02,Brazil,2020,02,NaN,NaN
2020-03,Brazil,2020,03,0.608867,NaN
2020-04,Brazil,2020,04,10.144900,NaN
2020-05,Brazil,2020,05,56.735452,NaN
2020-06,Brazil,2020,06,136.699552,NaN
...,...,...,...,...,...
2022-06,United States,2022,06,314.537467,67.019667
2022-07,United States,2022,07,364.569742,67.272258
2022-08,United States,2022,08,305.351581,67.557419


In [46]:
passenger_filtered_covid_data

,location,Year,Month,new_cases_smoothed_per_million,people_fully_vaccinated_per_hundred
date,,,,,
2020-01,Germany,2020,01,NaN,NaN
2020-02,Germany,2020,02,0.011621,NaN
2020-03,Germany,2020,03,18.258710,NaN
2020-04,Germany,2020,04,43.080800,NaN
2020-05,Germany,2020,05,9.641290,NaN
...,...,...,...,...,...
2022-03,United States,2022,03,107.913871,65.842258
2022-04,United States,2022,04,116.152833,66.299333
2022-05,United States,2022,05,266.394645,66.704516


In [54]:
passenger_filtered_covid_data.set_index('date', inplace = True)
passenger_filtered_covid_data = passenger_filtered_covid_data.drop(['2022-08','2022-09','2022-10'])
passenger_filtered_covid_data = passenger_filtered_covid_data.reset_index()
cargo_filtered_covid_data.set_index('date', inplace = True)
cargo_filtered_covid_data = cargo_filtered_covid_data.drop(['2022-08','2022-09','2022-10'])
cargo_filtered_covid_data = cargo_filtered_covid_data.reset_index()
cargo_filtered_covid_data

,date,location,Year,Month,new_cases_smoothed_per_million,people_fully_vaccinated_per_hundred
0,2020-02,Brazil,2020,02,NaN,NaN
1,2020-03,Brazil,2020,03,0.608867,NaN
2,2020-04,Brazil,2020,04,10.144900,NaN
3,2020-05,Brazil,2020,05,56.735452,NaN
4,2020-06,Brazil,2020,06,136.699552,NaN
...,...,...,...,...,...,...
148,2022-03,United States,2022,03,107.913871,65.842258
149,2022-04,United States,2022,04,116.152833,66.299333
150,2022-05,United States,2022,05,266.394645,66.704516
151,2022-06,United States,2022,06,314.537467,67.019667


In [56]:
merged_passengers = pd.merge(passenger_filtered_covid_data,passengers_pre_merge, left_on=('location','date'),right_on=('country_2_name','TIME_PERIOD'))
merged_cargo = pd.merge(cargo_filtered_covid_data,cargo_pre_merge, left_on=('location','date'),right_on=('country_2_name','TIME_PERIOD'))
merged_cargo

,date,location,Year,Month_x,new_cases_smoothed_per_million,people_fully_vaccinated_per_hundred,airp_country_2,TIME_PERIOD,Month_y,OBS_VALUE,OBS_FLAG,iso_3_country,country_2_name
0,2020-02,Brazil,2020,02,NaN,NaN,BR,2020-02,02,6228.1,0.0,BRA,Brazil
1,2020-03,Brazil,2020,03,0.608867,NaN,BR,2020-03,03,5274.1,0.0,BRA,Brazil
2,2020-04,Brazil,2020,04,10.144900,NaN,BR,2020-04,04,4406.7,0.0,BRA,Brazil
3,2020-05,Brazil,2020,05,56.735452,NaN,BR,2020-05,05,5642.5,0.0,BRA,Brazil
4,2020-06,Brazil,2020,06,136.699552,NaN,BR,2020-06,06,5448.5,0.0,BRA,Brazil
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,2022-03,United States,2022,03,107.913871,65.842258,US,2022-03,03,24360.1,0.0,USA,United States
149,2022-04,United States,2022,04,116.152833,66.299333,US,2022-04,04,22112.1,0.0,USA,United States
150,2022-05,United States,2022,05,266.394645,66.704516,US,2022-05,05,21808.0,0.0,USA,United States
151,2022-06,United States,2022,06,314.537467,67.019667,US,2022-06,06,19877.1,0.0,USA,United States


In [57]:
fig = px.line(merged_cargo, x='TIME_PERIOD', y= ['new_cases_smoothed_per_million','OBS_VALUE'] ,color='airp_country_2')
fig.show()


In [32]:

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Germany']['date'],y = merged_passengers[merged_passengers['location']== 'Germany']['OBS_VALUE'], name='Germany flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Germany']['date'],y = merged_passengers[merged_passengers['location']== 'Germany']['new_cases_smoothed_per_million'], name='Germany covid'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Italy']['date'],y = merged_passengers[merged_passengers['location']== 'Italy']['OBS_VALUE'], name='Italy flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Italy']['date'],y = merged_passengers[merged_passengers['location']== 'Italy']['new_cases_smoothed_per_million'], name='Italy covid'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'United States']['date'],y = merged_passengers[merged_passengers['location']== 'United States']['OBS_VALUE'], name='United States flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'United States']['date'],y = merged_passengers[merged_passengers['location']== 'United States']['new_cases_smoothed_per_million'], name='United States covid'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'United Kingdom']['date'],y = merged_passengers[merged_passengers['location']== 'United Kingdom']['OBS_VALUE'], name='United Kingdom flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'United Kingdom']['date'],y = merged_passengers[merged_passengers['location']== 'United Kingdom']['new_cases_smoothed_per_million'], name='United Kingdom covid'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Spain']['date'],y = merged_passengers[merged_passengers['location']== 'Spain']['OBS_VALUE'], name='Spain flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Spain']['date'],y = merged_passengers[merged_passengers['location']== 'Spain']['new_cases_smoothed_per_million'], name='Spain covid'),
    secondary_y=True,
)

options = {'Germany':'Germany','Italy':'Italy','United Stated':'United States','United Kingdom':'United Kingdom','Spain':'Spain'}
options2 = ['All','Germany','Italy','United States','United Kingdom','Spain','Covid only','Flight only']
visList = [
    [True,True,True,True,True,True,True,True,True,True],
    [True,True,False,False,False,False,False,False,False,False],
    [False,False,True,True,False,False,False,False,False,False],
    [False,False,False,False,True,True,False,False,False,False],
    [False,False,False,False,False,False,True,True,False,False],
    [False,False,False,False,False,False,False,False,True,True],
    [False,True,False,True,False,True,False,True,False,True],
    [True,False,True,False,True,False,True,False,True,False]]
buttons = []
for i, g in enumerate(options2):
    button = dict(label =  g,
    method = 'restyle',
    args = ['visible',visList[i]])
    buttons.append(button)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Number of passengers',
    title_text='Flight passengers compared to the covid cases per million',
    updatemenus=[
        dict(
            type='dropdown',
            direction='right',
            buttons = buttons,
            x = 0.13,
            y = 1.16,
            bordercolor = 'grey'   
        )
    ],
)
fig.update_yaxes(title_text='Covid cases per million people', secondary_y=True)
fig.show()

In [61]:

fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'China']['date'],y = merged_cargo[merged_cargo['location']== 'China']['OBS_VALUE'], name='China flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'China']['date'],y = merged_cargo[merged_cargo['location']== 'China']['new_cases_smoothed_per_million'], name='China covid'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'United Arab Emirates']['date'],y = merged_cargo[merged_cargo['location']== 'United Arab Emirates']['OBS_VALUE'], name='United Arab Emirates flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'United Arab Emirates']['date'],y = merged_cargo[merged_cargo['location']== 'United Arab Emirates']['new_cases_smoothed_per_million'], name='United Arab Emirates covid'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'United States']['date'],y = merged_cargo[merged_cargo['location']== 'United States']['OBS_VALUE'], name='United States flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'United States']['date'],y = merged_cargo[merged_cargo['location']== 'United States']['new_cases_smoothed_per_million'], name='United States covid'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'Brazil']['date'],y = merged_cargo[merged_cargo['location']== 'Brazil']['OBS_VALUE'], name='Brazil flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'Brazil']['date'],y = merged_cargo[merged_cargo['location']== 'Brazil']['new_cases_smoothed_per_million'], name= 'Brazil covid'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'Qatar']['date'],y = merged_cargo[merged_cargo['location']== 'Qatar']['OBS_VALUE'], name='Qatar flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'Qatar']['date'],y = merged_cargo[merged_cargo['location']== 'Qatar']['new_cases_smoothed_per_million'], name='Qatar covid'),
    secondary_y=True,
)

options2 = ['All','China','United Arab Emirates','United States','Brazil','Qatar','Covid only','Flight only']
visList = [
    [True,True,True,True,True,True,True,True,True,True],
    [True,True,False,False,False,False,False,False,False,False],
    [False,False,True,True,False,False,False,False,False,False],
    [False,False,False,False,True,True,False,False,False,False],
    [False,False,False,False,False,False,True,True,False,False],
    [False,False,False,False,False,False,False,False,True,True],
    [False,True,False,True,False,True,False,True,False,True],
    [True,False,True,False,True,False,True,False,True,False]]
buttons = []
for i, g in enumerate(options2):
    button = dict(label =  g,
    method = 'restyle',
    args = ['visible',visList[i]])
    buttons.append(button)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Transported cargo [Tonnes]',
    title_text='Flight passengers compared to the covid cases per million',
    updatemenus=[
        dict(
            type='dropdown',
            direction='right',
            buttons = buttons,
            x = 0.17,
            y = 1.16,
            bordercolor = 'grey'   
        )
    ],
)
fig.update_yaxes(title_text='Covid cases per million people', secondary_y=True)
fig.show()

In [64]:

fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Germany']['date'],y = merged_passengers[merged_passengers['location']== 'Germany']['OBS_VALUE'], name='Germany flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Germany']['date'],y = merged_passengers[merged_passengers['location']== 'Germany']['people_fully_vaccinated_per_hundred'], name='Germany vaccinations'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Italy']['date'],y = merged_passengers[merged_passengers['location']== 'Italy']['OBS_VALUE'], name='Italy flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Italy']['date'],y = merged_passengers[merged_passengers['location']== 'Italy']['people_fully_vaccinated_per_hundred'], name='Italy vaccinations'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'United States']['date'],y = merged_passengers[merged_passengers['location']== 'United States']['OBS_VALUE'], name='United States flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'United States']['date'],y = merged_passengers[merged_passengers['location']== 'United States']['people_fully_vaccinated_per_hundred'], name='United States vaccinations'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'United Kingdom']['date'],y = merged_passengers[merged_passengers['location']== 'United Kingdom']['OBS_VALUE'], name='United Kingdom flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'United Kingdom']['date'],y = merged_passengers[merged_passengers['location']== 'United Kingdom']['people_fully_vaccinated_per_hundred'], name='United Kingdom vaccinations'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Spain']['date'],y = merged_passengers[merged_passengers['location']== 'Spain']['OBS_VALUE'], name='Spain flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_passengers[merged_passengers['location']== 'Spain']['date'],y = merged_passengers[merged_passengers['location']== 'Spain']['people_fully_vaccinated_per_hundred'], name='Spain vaccinations'),
    secondary_y=True,
)

options = {'Germany':'Germany','Italy':'Italy','United Stated':'United States','United Kingdom':'United Kingdom','Spain':'Spain'}
options2 = ['All','Germany','Italy','United States','United Kingdom','Spain','Vaccinations only','Flight only']
visList = [
    [True,True,True,True,True,True,True,True,True,True],
    [True,True,False,False,False,False,False,False,False,False],
    [False,False,True,True,False,False,False,False,False,False],
    [False,False,False,False,True,True,False,False,False,False],
    [False,False,False,False,False,False,True,True,False,False],
    [False,False,False,False,False,False,False,False,True,True],
    [False,True,False,True,False,True,False,True,False,True],
    [True,False,True,False,True,False,True,False,True,False]]
buttons = []
for i, g in enumerate(options2):
    button = dict(label =  g,
    method = 'restyle',
    args = ['visible',visList[i]])
    buttons.append(button)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Number of passengers',
    title_text='Flight passengers compared to the percentage of vaccinated people',
    updatemenus=[
        dict(
            type='dropdown',
            direction='right',
            buttons = buttons,
            x = 0.13,
            y = 1.16,
            bordercolor = 'grey'   
        )
    ],
)
fig.update_yaxes(title_text='Fully vaccinated people [%]' , secondary_y=True)
fig.show()

In [68]:

fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'China']['date'],y = merged_cargo[merged_cargo['location']== 'China']['OBS_VALUE'], name='China flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'China']['date'],y = merged_cargo[merged_cargo['location']== 'China']['people_fully_vaccinated_per_hundred'], name='China vaccinations'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'United Arab Emirates']['date'],y = merged_cargo[merged_cargo['location']== 'United Arab Emirates']['OBS_VALUE'], name='United Arab Emirates flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'United Arab Emirates']['date'],y = merged_cargo[merged_cargo['location']== 'United Arab Emirates']['people_fully_vaccinated_per_hundred'], name='United Arab Emirates vaccinations'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'United States']['date'],y = merged_cargo[merged_cargo['location']== 'United States']['OBS_VALUE'], name='United States flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'United States']['date'],y = merged_cargo[merged_cargo['location']== 'United States']['people_fully_vaccinated_per_hundred'], name='United States vaccinations'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'Brazil']['date'],y = merged_cargo[merged_cargo['location']== 'Brazil']['OBS_VALUE'], name='Brazil flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'Brazil']['date'],y = merged_cargo[merged_cargo['location']== 'Brazil']['people_fully_vaccinated_per_hundred'], name= 'Brazil vaccinations'),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'Qatar']['date'],y = merged_cargo[merged_cargo['location']== 'Qatar']['OBS_VALUE'], name='Qatar flight'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x = merged_cargo[merged_cargo['location']== 'Qatar']['date'],y = merged_cargo[merged_cargo['location']== 'Qatar']['people_fully_vaccinated_per_hundred'], name='Qatar vaccinations'),
    secondary_y=True,
)

options2 = ['All','China','United Arab Emirates','United States','Brazil','Qatar','Vaccinations only','Flight only']
visList = [
    [True,True,True,True,True,True,True,True,True,True],
    [True,True,False,False,False,False,False,False,False,False],
    [False,False,True,True,False,False,False,False,False,False],
    [False,False,False,False,True,True,False,False,False,False],
    [False,False,False,False,False,False,True,True,False,False],
    [False,False,False,False,False,False,False,False,True,True],
    [False,True,False,True,False,True,False,True,False,True],
    [True,False,True,False,True,False,True,False,True,False]]
buttons = []
for i, g in enumerate(options2):
    button = dict(label =  g,
    method = 'restyle',
    args = ['visible',visList[i]])
    buttons.append(button)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Transported cargo [Tonnes]',
    title_text='Flight passengers compared to the percentage of people vaccinated',
    updatemenus=[
        dict(
            type='dropdown',
            direction='right',
            buttons = buttons,
            x = 0.15,
            y = 1.16,
            bordercolor = 'grey'   
        )
    ],
)
fig.update_yaxes(title_text='Fully vaccinated people [%]', secondary_y=True)
fig.show()